### Session-Stopper

In [28]:
import time
for i in range(50):
  print('Stop: ', i)
  time.sleep(30*60)

In [3]:
!pip install -q pymap3d
!unzip -q /content/optical-edge-dev.zip
!unzip -q sample-dataset-xss.zip

replace optical-edge-dev/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import os
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import random
import numpy as np
from math import atan2, asin
import pandas as pd
from IPython.display import display
from pprint import pprint
from PIL import Image
import time
import datetime
import time
import sys
from pymap3d import geodetic2enu, enu2geodetic
import warnings
warnings.filterwarnings('ignore')

In [5]:
sys.path.append('/content/optical-edge-dev')
import pose_est
from pose_est import *
import vo
from vo import Stitcher
from vo import FeatureLocalization

In [25]:
import Utils
from Utils import jetson_timestamp_to_utc, closest_ea, closest_t, get_unix_timestamps, trans_vec_to_lla, EAE

In [24]:
import importlib
importlib.reload(vo)
importlib.reload(Utils)

<module 'Utils' from '/content/Utils.py'>

In [8]:
 sample_dataset_path = r"/content/sample-dataset-xss"

In [9]:
sample_dir = os.listdir(sample_dataset_path)
sample_dir = [path for path in sample_dir if path.endswith('.tiff')]
print('No. of Samples: ', len(sample_dir))


def key_fn(item):
  return item.split('_')[2]

sample_dir = sorted(sample_dir, key=key_fn, reverse=False)
print(sample_dir[:2])

No. of Samples:  5
['img_E240_360566645427_1679940008_58138883.tiff', 'img_E240_360900654894_1679940008_391646443.tiff']


In [10]:
print(len(sample_dir))
timestamps_list = ['_'.join(nm.split('_')[3:5])[:-5] for nm in sample_dir]
utc_timestamps_list = list(map(jetson_timestamp_to_utc, timestamps_list))

print(timestamps_list)
print(utc_timestamps_list)

flight_obj = {}
rev_flight_obj = {}

for time, path in zip(utc_timestamps_list, sample_dir):
  flight_obj[time] = path
  rev_flight_obj[path] = time

unix_timestamps_list = ['_'.join(nm.split('_')[3:5])[:-5] for nm in sample_dir]
unix_timestamps_list = list(map(get_unix_timestamps, timestamps_list))
print('Timestamps: ', unix_timestamps_list[:3], ', Timestamps-size: ', len(unix_timestamps_list))

# print()
# print('Time-Wise Path Storages -')
# print()
# pprint(flight_obj)


5
['1679940008_58138883', '1679940008_391646443', '1679940008_724268128', '1679940009_58016653', '1679940009_391413395']
['2023-03-27 18:00:08.058139', '2023-03-27 18:00:08.391646', '2023-03-27 18:00:08.724268', '2023-03-27 18:00:09.058017', '2023-03-27 18:00:09.391413']
Timestamps:  [1679940008.058139, 1679940008.391646, 1679940008.724268] , Timestamps-size:  5


In [11]:
combine_ins_path_all = r"/content/combine_ins_E240_1679939655_time.csv"
combine_ins_path_time = r"/content/ins_E240_1679939655_time.csv"

dtypes = {' Jetson Time': np.float64}
df_ins_data_time = pd.read_csv(combine_ins_path_time, float_precision='high', dtype=dtypes)
df_ins_data_all = pd.read_csv(combine_ins_path_all)

display(df_ins_data_time.head(3))
display(df_ins_data_all.head(3))
df_ins_data_time[' Jetson Time'][0]

,Packet Count,Jetson Time
0,0,1.679940e+09
1,1,1.679940e+09
2,2,1.679940e+09


,Packet Count,UTC Time,Acc_SensorX,Acc_SensorY,Acc_SensorZ,Gyr_SensorX,Gyr_SensorY,Gyr_SensorZ,Mag_SensorX,Mag_SensorY,...,RawAccZ,SampleTimeFine,LatitudeRaw,LongitudeRaw,Height,HeightMSL,nanoseconds,Pressure,Packet Countextra,Jetson Time
0,0,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,407364,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.679940e+09
1,1,33,-0.33,0.3,9.8,-0.02,0.0,0.0,-0.6,-0.39,...,NaN,407464,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.679940e+09
2,2,33,-0.33,0.3,9.8,-0.02,0.0,0.0,-0.6,-0.39,...,NaN,407564,NaN,NaN,NaN,NaN,NaN,101156.0,2.0,1.679940e+09


1679939655.8286555

In [12]:
j_col = df_ins_data_time.columns[-1]
p_col = df_ins_data_time.columns[0]

print('Columns of Interest: ', j_col, ', ', p_col)

Columns of Interest:   Jetson Time ,  Packet Count


In [13]:
ins_jetson = list(df_ins_data_time[j_col].dropna())
ins_jetson = [str(time).split('.')[0] + '_' + '0' for time in ins_jetson]
utc_ins_jetson = list(map(jetson_timestamp_to_utc, ins_jetson))
ins_flight_obj = {}
for unix_time, utc_time in zip(ins_jetson, utc_ins_jetson):
  ins_flight_obj[utc_time] = unix_time

In [14]:
image_size = (1200, 1920)
num_frames = len(sample_dir)
print('No. of frames:', num_frames)
fx, fy, cx, cy = [718.8560, 718.8560, 607.1928, 185.2157]
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

No. of frames: 5


In [15]:
import numpy as np

sensor_width = 29.3  # mm
sensor_height = 29  # mm

image_width = 1920  # in pixels
image_height = 1080  # in pixels

pixel_size_x = sensor_width / image_width
pixel_size_y = sensor_height / image_height

scale_factor_x = pixel_size_x / fx  # m/pixel
scale_factor_y = pixel_size_y / fy  # m/pixel

scale_factor = (scale_factor_x + scale_factor_y) / 2

print('Scale-Factor: ', scale_factor)

Scale-Factor:  2.9291171332310313e-05


In [26]:
import time

initial_frame_timestamp = sample_dir[0]
print('initial_frame_timestamp: ', initial_frame_timestamp)

# ref_coordinates = [12.9, 80.2, 6.7]
df_data = [df_ins_data_time, df_ins_data_all]
ini_ea = closest_ea(initial_frame_timestamp, rev_flight_obj, df_data)
ini_coordinates = closest_t(initial_frame_timestamp, rev_flight_obj, df_data)

ini_frame_path = os.path.join(sample_dataset_path, sample_dir[0])
ini_frame = np.asarray(Image.open(ini_frame_path))

df_OF_cols = ['Timestamps', 'Roll_N', 'Pitch_E', 'Yaw_U', 'Latitude', 'Longitude', 'Altitude']
roll_list, pitch_list, yaw_list, lat_list, lon_list, alt_list = [], [], [], [], [], []

ft = time.time()

for i in tqdm(range(num_frames), desc="Processing frames", unit="frame"):

  roll_list.append(ini_ea[0])
  pitch_list.append(ini_ea[1])
  yaw_list.append(ini_ea[-1])
  lat_list.append(ini_coordinates[0])
  lon_list.append(ini_coordinates[1])
  alt_list.append(ini_coordinates[-1])

  if (i + 1) >= num_frames:
    break

  pass

  next_frame_path = os.path.join(sample_dataset_path, sample_dir[i + 1])
  next_frame = np.asarray(Image.open(next_frame_path))

  eae = EAE(ini_frame, next_frame, FeatureLocalization)
  one_frame_R, one_frame_t, euler_angle_rad, _ = eae.run(is_store = True)

  pred_ins_euler_angle = [(prev_ea + delta_ea) for delta_ea, prev_ea in zip(list(euler_angle_rad.values()), ini_ea)]
  lat, lon, alt = trans_vec_to_lla(one_frame_t, one_frame_R, scale_factor, ini_ea, ini_coordinates)

  ini_frame = next_frame
  ini_ea = pred_ins_euler_angle
  ini_coordinates = [lat, lon, alt]


df_of = pd.DataFrame(columns = df_OF_cols)

data = {
    'Timestamps': unix_timestamps_list,
    'Roll_N': roll_list,
    'Pitch_E': pitch_list,
    'Yaw_U': yaw_list,
    'Latitude': lat_list,
    'Longitude': lon_list,
    'Altitude': alt_list
}


df_of = pd.DataFrame(data)
display(df_of.head(3))

lt = time.time()

print('Execution-time (UNIX): ', (lt - ft))

initial_frame_timestamp:  img_E240_360566645427_1679940008_58138883.tiff


Processing frames:  80%|████████  | 4/5 [00:19<00:04,  4.89s/frame]


,Timestamps,Roll_N,Pitch_E,Yaw_U,Latitude,Longitude,Altitude
0,1.679940e+09,-1.840000,8.940000,-74.950000,12.791755,80.1884,13.530000
1,1.679940e+09,-1.840003,8.940004,-74.972823,12.791755,80.1884,13.530011
2,1.679940e+09,-1.840009,8.940011,-74.994042,12.791755,80.1884,13.530025


Execution-time (UNIX):  19.576204299926758


In [27]:
df_of.to_csv('optical_frame_parameters_strip_E240_1679940007.csv', index=False)